In [25]:
# Dependencies
import tweepy
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import style
import time
#style.use('ggplot')

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Twitter API Keys
from config import (consumer_key, 
                    consumer_secret, 
                    access_token, 
                    access_token_secret)
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [26]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [27]:
movie_df = pd.read_csv("../data/sample_movies_with_twitter.csv")

movie_df.head()

,row,rank,movie_title,studio,total_gross,total_theater,opening_weekend_gross,opening_theater_count,release_date,close_date,twitter1,twitter2,twitter3,twitter4,twitter5
0,2.0,57.0,Mission: Impossible - Fallout,Par.,"$77,526,635","4,386","$61,236,534","4,386",7/27/2018,-,MissionImpossible,MissionImpossibleFallout,NaN,NaN,NaN
1,3.0,84.0,Teen Titans Go! To The Movies,WB,"$13,640,501","3,188","$10,411,189","3,188",7/27/2018,-,TeenTitansGOMovie,NaN,NaN,NaN,NaN
2,5.0,22.0,Mamma Mia! Here We Go Again,Uni.,"$60,227,525","3,514","$34,952,180","3,317",7/20/2018,-,MammaMia2,NaN,NaN,NaN,NaN
3,6.0,28.0,The Equalizer 2,Sony,"$54,241,345","3,388","$36,011,640","3,388",7/20/2018,-,TheEqualizer2,NaN,NaN,NaN,NaN
4,7.0,76.0,Unfriended: Dark Web,BH Tilt,"$6,375,430","1,547","$3,653,035","1,546",7/20/2018,-,UnfriendedMovie,UnfriendedDarkWeb,NaN,NaN,NaN


In [28]:
# loop through csv and get tweets for the movie
movieTitle = []
tweetText = []
tweetCreated = []
lang = "en"

# initialize sentiment score for each movie
compound_list = []
positive_list = []
negative_list = []
neutral_list = []

counter = 1


for i in movie_df.index:
        tweeterName1 = movie_df.loc[i,'twitter1']
        movie_title = movie_df.loc[i,'movie_title']
        print(tweeterName1)
        
        oldest_tweet = None
        
        for x in range(1, 16):

            # per twitter -  Requests / 15-min window (app auth) - 450
            # so pull 400 tweets, then sleep for 15 minutes before pulling the next set of 400
            if (counter % 5 == 0 ):
                time.sleep(900)
                
            # search for tweets on the movie
            public_tweets = api.search(tweeterName1, 
                                       count=100, 
                                       result_type="recent", 
                                       max_id=oldest_tweet) 
            counter += 1
            print (counter)
            
            for tweet in public_tweets["statuses"]:
                # only get sentiments of English tweets
                if (tweet["user"]["lang"] == lang) :
                    tweet_id = tweet["id"]
                    movieTitle.append(movie_title)
                    tweetText.append(tweet["text"])
                    tweetCreated.append(tweet["created_at"])
                    results = analyzer.polarity_scores(tweet["text"])
            
                    compound = results["compound"]
                    pos = results["pos"]
                    neu = results["neu"]
                    neg = results["neg"]
            
                    # add scores to their perspective LISTs
                    compound_list.append(compound)
                    positive_list.append(pos)
                    negative_list.append(neg)
                    neutral_list.append(neu)
            
            oldest_tweet = tweet_id - 1
            

MissionImpossible
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
TeenTitansGOMovie
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
MammaMia2
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
TheEqualizer2


TweepError: Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Read timed out.

In [29]:
print(counter)

46


In [ ]:
#public_tweets = api.search("HanSolo", rpp=1000)
#tweetText = []
#tweetCreated = []

#for tweet in public_tweets["statuses"]:
#    tweetText.append(tweet["text"])
#    tweetCreated.append(tweet["created_at"])


In [30]:
movieTweets ={"MovieTitle" : movieTitle,
              "TweetID" : tweet_id, 
              "TweetText": tweetText,
              "TweetCreated": tweetCreated,
              "CompoundScore" : compound_list,
              "positive_list": positive_list,
              "negative_list" : negative_list,
              "neutral_list" : neutral_list}
movieTweets_df = pd.DataFrame(movieTweets)
movieTweets_df.head()
len(movieTweets_df)
movieTweets_df.to_csv("../data/MovieTweets.csv", index=False)
